# Account - AVR Guest Experience PROD

## Database Summary

In [1]:
Import-Module AWS.Tools.Common, AWS.Tools.DynamoDBv2, AWS.Tools.RDS
Import-Module '/Users/conrad.gauntlett/WorkArea/Repos/DBA_MISC/PowerShell/Modules/DevOpsToolkit'

$Account            = 'DevOps-AVR-Guest-Experience-PROD'
$RDSSummaryColumns  = 'AvailabilityZone', 'DBSnapshotIdentifier', 'SnapshotCreateTime', 'SnapshotType', 'Status'
$Profiles           = Get-AWSCredentials -ListProfileDetail | Where-Object { $_.ProfileName -match $Account }
$Regions            = "eu-central-1" , "eu-west-2" , "eu-west-1", "us-east-1"
$Summary            = Get-DoAWSDBInformation $Profiles $Regions
$Summary | Format-Table -AutoSize


Account                   AccountID    Region    RDSInstances DynamoDBTables
-------                   ---------    ------    ------------ --------------
AVR-Guest-Experience-PROD 957578851978 eu-west-2            1              2



### DynamoDB Tables

In [2]:
$ddbTables = Get-DoDDBTableInformation $Profiles $Regions 
$ddbTables | Format-Table -AutoSize


Region    TableName       TableBackup CreationDateTime      ItemCount TableSizeMB
------    ---------       ----------- ----------------      --------- -----------
eu-west-2 terraform-lock        False 3/16/2021 2:55:01 PM         49 0 MB
eu-west-2 terraform-locks       False 10/19/2021 1:14:49 PM        68 0 MB



### RDS Instances

In [3]:
$RDS        = Get-DoRDSDBSummary $Profiles $Regions 
$Snapshots  = Get-DoRDSSnapshotSummary $Profiles $Regions
$RDS | Format-Table -AutoSize


Region    DBInstanceIdentifier Engine  Version InstanceCreateTime   Snapshots Retention
------    -------------------- ------  ------- ------------------   --------- ---------
eu-west-2 places-api           mariadb 10.6.16 4/11/2023 3:36:31 PM         9         7



#### Snapshots

##### places-api

In [4]:
$DB = 'places-api'
$Snapshots | Where-Object {$_.DBInstanceIdentifier -eq $DB} | Select-Object $RDSSummaryColumns |
Sort-Object SnapshotCreateTime -Descending  |Format-Table -AutoSize


AvailabilityZone DBSnapshotIdentifier            SnapshotCreateTime   SnapshotType Status
---------------- --------------------            ------------------   ------------ ------
eu-west-2b       rds:places-api-2024-07-15-03-13 7/15/2024 4:13:22 AM automated    available
eu-west-2b       rds:places-api-2024-07-14-03-13 7/14/2024 4:13:54 AM automated    available
eu-west-2b       rds:places-api-2024-07-13-03-12 7/13/2024 4:12:54 AM automated    available
eu-west-2b       rds:places-api-2024-07-12-03-13 7/12/2024 4:14:01 AM automated    available
eu-west-2b       rds:places-api-2024-07-11-03-12 7/11/2024 4:12:36 AM automated    available
eu-west-2b       rds:places-api-2024-07-10-03-12 7/10/2024 4:12:53 AM automated    available
eu-west-2b       rds:places-api-2024-07-09-03-14 7/9/2024 4:14:26 AM  automated    available
eu-west-2b       rds:places-api-2024-07-08-03-13 7/8/2024 4:13:36 AM  automated    available
eu-west-2b       rds:places-api-2024-07-07-03-13 7/7/2024 4:13:19 AM  autom